In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sci_opt

from pprint import pprint
from sklearn.preprocessing import StandardScaler
# Set some display options for Pandas.
%config InlineBackend.figure_format ='retina'
pd.set_option('display.max_colwidth', None)
pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_columns', None)

In [2]:
## Fetching S&P 500 Companies:
sp500_companies = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500_companies['Symbol'] = sp500_companies['Symbol'].replace({'BRK.B': 'BRK-B', 'BF.B': 'BF-B'})

In [3]:
sp500_companies

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [4]:
# Dropping NaN values
sp500_companies.dropna(inplace=True)
## Getting Unique Sectors and Dates:
unique_sectors = sp500_companies['GICS Sector'].unique()
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=90)).strftime('%Y-%m-%d')
unique_sectors_df = pd.DataFrame(unique_sectors)
print(unique_sectors_df)

                         0
0              Industrials
1              Health Care
2   Information Technology
3                Utilities
4               Financials
5                Materials
6   Consumer Discretionary
7              Real Estate
8   Communication Services
9         Consumer Staples
10                  Energy


In [5]:
number_of_sectors = len(unique_sectors)
len(unique_sectors)

11

In [6]:
## Fetching Historical Prices and Calculating Averages:
## The code fetches historical closing prices for each company in each sector and calculates the average closing price for the sector.
# Loop through each sector and download closing prices
all_avg_prices = []
all_dates = []
all_sectors = []
for sector in unique_sectors:
    companies = sp500_companies[sp500_companies['GICS Sector'] == sector]['Symbol'].tolist()
    prices = yf.download(companies, start=start_date, end=end_date)['Adj Close']
    avg_prices = prices.mean(axis=1)
    # Append the data
    all_avg_prices.extend(avg_prices.tolist())
    all_dates.extend([date.date() for date in prices.index])
    all_sectors.extend([sector] * len(avg_prices))

# Create a DataFrame
sp500_sectors_avg_prices = pd.DataFrame({
    'Date': all_dates,
    'Avg_Close': all_avg_prices,
    'Sector': all_sectors
})

#sp500_sectors_avg_prices = sp500_sectors_avg_prices.pivot(index='Date', columns='Sector', values='Avg_Close')
# Reverse the order of the DataFrame by dates
sp500_sectors_avg_prices = sp500_sectors_avg_prices[::-1]

print(sp500_sectors_avg_prices.head(100))

[*********************100%%**********************]  78 of 78 completed
[*********************100%%**********************]  64 of 64 completed
[*********************100%%**********************]  64 of 64 completed
[*********************100%%**********************]  30 of 30 completed
[*********************100%%**********************]  72 of 72 completed
[*********************100%%**********************]  28 of 28 completed
[*********************100%%**********************]  53 of 53 completed
[*********************100%%**********************]  31 of 31 completed
[*********************100%%**********************]  22 of 22 completed
[*********************100%%**********************]  38 of 38 completed
[*********************100%%**********************]  23 of 23 completed
           Date   Avg_Close            Sector
659  2024-02-01   86.093043            Energy
658  2024-01-31   86.108696            Energy
657  2024-01-30   87.736087            Energy
656  2024-01-29   86.452325        

In [7]:
sp500_sectors_avg_prices = sp500_sectors_avg_prices.pivot(
    index='Date',
    columns='Sector',
    values='Avg_Close'
)
print(sp500_sectors_avg_prices.head())


Sector      Communication Services  Consumer Discretionary  Consumer Staples     Energy  Financials  Health Care  Industrials  Information Technology   Materials  Real Estate  Utilities
Date                                                                                                                                                                                     
2023-11-06              107.379353              384.055126         96.841980  86.775185  143.068032   212.388183   176.073544              224.918651  124.745339   112.064622  63.856103
2023-11-07              107.619845              388.411453         97.102875  84.800603  142.851109   212.798488   176.089518              226.842666  122.782611   111.098683  63.352058
2023-11-08              107.981790              386.795342         96.718323  83.600292  143.246391   211.858423   176.464457              228.190706  123.598667   111.901380  62.954094
2023-11-09              107.596075              384.655803         96.

In [8]:
sp500_sectors_avg_prices.dropna(inplace=True)
print(sp500_sectors_avg_prices.isnull().sum())

Sector
Communication Services    0
Consumer Discretionary    0
Consumer Staples          0
Energy                    0
Financials                0
Health Care               0
Industrials               0
Information Technology    0
Materials                 0
Real Estate               0
Utilities                 0
dtype: int64


In [9]:
sp500_sectors_avg_prices.to_excel('sp500_sectors_avg_prices.xlsx', index=True)
# Calculate the Log of returns.
log_return = np.log(1 +sp500_sectors_avg_prices.pct_change().iloc[::-1])

# Drop rows with negative values
log_return = log_return[(log_return > 0).all(axis=1)]


# Generate Random Weights.
random_weights = np.array(np.random.random(number_of_sectors))

# Generate the Rebalance Weights, these should equal 1.
rebalance_weights = random_weights / np.sum(random_weights)
print('Log Returns:')
print(log_return.head())

Log Returns:
Sector      Communication Services  Consumer Discretionary  Consumer Staples    Energy  Financials  Health Care  Industrials  Information Technology  Materials  Real Estate  Utilities
Date                                                                                                                                                                                   
2023-12-26                0.006612                0.001971          0.005255  0.010995    0.003017     0.004256     0.007121                0.007045   0.004890     0.007888   0.006605
2023-12-21                0.009171                0.010540          0.006759  0.004001    0.011064     0.015268     0.012647                0.014892   0.010197     0.010102   0.002624
2023-12-19                0.010264                0.010588          0.005005  0.011440    0.005798     0.008706     0.006324                0.002996   0.008073     0.005542   0.005890
2023-12-13                0.017362                0.016282         

In [10]:
print((sp500_sectors_avg_prices == 0).sum())

Sector
Communication Services    0
Consumer Discretionary    0
Consumer Staples          0
Energy                    0
Financials                0
Health Care               0
Industrials               0
Information Technology    0
Materials                 0
Real Estate               0
Utilities                 0
dtype: int64


In [11]:
log_return.to_excel('log_return.xlsx', index=True)
# Calculate the percentage of negative log returns
percentage_negative_returns = (log_return < 0).mean().mean() * 100

print(f"Percentage of negative log returns: {percentage_negative_returns:.2f}%")

Percentage of negative log returns: 0.00%


In [12]:
# Calculate the Expected Returns, annualize it by multiplying it by `252`.
risk_free_rate = .01
exp_ret = np.sum(((log_return.mean()-risk_free_rate) * rebalance_weights) * 252)

# Calculate the Expected Volatility, annualize it by multiplying it by `252`.
exp_vol = np.sqrt(
np.dot(
    rebalance_weights.T,
    np.dot(
        log_return.cov() * 252,
        rebalance_weights
    )
)
)

# Calculate the Sharpe Ratio.
sharpe_ratio = exp_ret / exp_vol

# Put the weights into a data frame to see them better.
weights_df = pd.DataFrame(data={
'random_weights': random_weights,
'rebalance_weights': rebalance_weights
})
print('')
print('='*80)
print('PORTFOLIO WEIGHTS:')
print('-'*80)
print(weights_df)
print('-'*80)

# Do the same with the other metrics.
metrics_df = pd.DataFrame(data={
    'Expected Portfolio Returns': exp_ret,
    'Expected Portfolio Volatility': exp_vol,
    'Portfolio Sharpe Ratio': sharpe_ratio
}, index=[0])

print('')
print('='*80)
print('PORTFOLIO METRICS:')
print('-'*80)
print(metrics_df)
print('-'*80)


PORTFOLIO WEIGHTS:
--------------------------------------------------------------------------------
    random_weights  rebalance_weights
0         0.560410           0.120796
1         0.725787           0.156443
2         0.173865           0.037476
3         0.624587           0.134629
4         0.262699           0.056625
5         0.219882           0.047395
6         0.267079           0.057569
7         0.610351           0.131561
8         0.235035           0.050662
9         0.571922           0.123277
10        0.387694           0.083567
--------------------------------------------------------------------------------

PORTFOLIO METRICS:
--------------------------------------------------------------------------------
   Expected Portfolio Returns  Expected Portfolio Volatility  Portfolio Sharpe Ratio
0                    0.760516                       0.114729                6.628787
--------------------------------------------------------------------------------


In [13]:
# Initialize the components, to run a Monte Carlo Simulation.

# We will run 5000 iterations.
num_of_portfolios = 20000

# Prep an array to store the weights as they are generated, 5000 iterations for each of our 4 symbols.
all_weights = np.zeros((num_of_portfolios, number_of_sectors))

# Prep an array to store the returns as they are generated, 5000 possible return values.
ret_arr = np.zeros(num_of_portfolios)

# Prep an array to store the volatilities as they are generated, 5000 possible volatility values.
vol_arr = np.zeros(num_of_portfolios)

# Prep an array to store the sharpe ratios as they are generated, 5000 possible Sharpe Ratios.
sharpe_arr = np.zeros(num_of_portfolios)

# Start the simulations.
for ind in range(num_of_portfolios):

    # First, calculate the weights.
    weights = np.array(np.random.random(number_of_sectors))
    weights = weights / np.sum(weights)

    # Add the weights, to the `weights_arrays`.
    all_weights[ind, :] = weights

    # Calculate the expected log returns, and add them to the `returns_array`.
    ret_arr[ind] = np.sum(((log_return.mean()-risk_free_rate) * weights) * 252)

    # Calculate the volatility, and add them to the `volatility_array`.
    vol_arr[ind] = np.sqrt(
        np.dot(weights.T, np.dot(log_return.cov() * 252, weights)))


    # Calculate the Sharpe Ratio and Add it to the `sharpe_ratio_array`.
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

# Let's create our "Master Data Frame", with the weights, the returns, the volatility, and the Sharpe Ratio
simulations_data = [ret_arr, vol_arr, sharpe_arr, all_weights]

# Create a DataFrame from it, then Transpose it so it looks like our original one.
simulations_df = pd.DataFrame(data=simulations_data).T

# Give the columns the Proper Names.
simulations_df.columns = [
    'Returns',
    'Volatility',
    'Sharpe Ratio',
    'Portfolio Weights'
]

# Make sure the data types are correct, we don't want our floats to be strings.
simulations_df = simulations_df.infer_objects()

# Print out the results.
print('')
print('='*80)
print('SIMULATIONS RESULT:')
print('-'*80)

# Print PORTFOLIO WEIGHTS
print('='*80)
print('PORTFOLIO WEIGHTS:')
print('-'*80)
print(simulations_df['Portfolio Weights'].head())
print('-'*80)

# Print PORTFOLIO METRICS
print('='*80)
print('PORTFOLIO METRICS:')
print('-'*80)
print(simulations_df[['Returns', 'Volatility', 'Sharpe Ratio']].head())
print('-'*80)


SIMULATIONS RESULT:
--------------------------------------------------------------------------------
PORTFOLIO WEIGHTS:
--------------------------------------------------------------------------------
0    [0.15590447184185768, 0.057384483665105124, 0.0781244636046349, 0.05575516447099274, 0.10186732518018163, 0.039045293106263805, 0.09102103948435449, 0.004378654832128958, 0.12108445370491619, 0.12114030864800975, 0.17429434146155467]
1          [0.0406975440839412, 0.1287752558441318, 0.10414383001281226, 0.00898338858973777, 0.166162598037402, 0.0814400365567108, 0.11634034923300363, 0.1629342630091322, 0.13277675723275084, 0.037334695933648604, 0.020411281466729055]
2        [0.0743085140541715, 0.16939988988021168, 0.09520914477623667, 0.02099204648845927, 0.03311527881131096, 0.1445137310238473, 0.09057225267335094, 0.04235756092519942, 0.16168062007967807, 0.10529748225493517, 0.06255347903259902]
3     [0.09797647858772943, 0.008321053928051525, 0.05919647671061805, 0.06792984

In [14]:
# Return the Max Sharpe Ratio from the run.
max_sharpe_ratio = simulations_df.loc[simulations_df['Sharpe Ratio'].idxmax()]

# Return the Min Volatility from the run.
min_volatility = simulations_df.loc[simulations_df['Volatility'].idxmin()]

print('\n' + '='*80)
print('MAX SHARPE RATIO:')
print('-'*80)
print(f'Returns: {max_sharpe_ratio["Returns"]}')
print(f'Volatility: {max_sharpe_ratio["Volatility"]}')
print(f'Sharpe Ratio: {max_sharpe_ratio["Sharpe Ratio"]}')
print(f'Portfolio Weights: {max_sharpe_ratio["Portfolio Weights"]}')
print('-'*80)

print('\n' + '='*80)
print('MIN VOLATILITY:')
print('-'*80)
print(f'Returns: {min_volatility["Returns"]}')
print(f'Volatility: {min_volatility["Volatility"]}')
print(f'Sharpe Ratio: {min_volatility["Sharpe Ratio"]}')
print(f'Portfolio Weights: {min_volatility["Portfolio Weights"]}')
print('-'*80)



MAX SHARPE RATIO:
--------------------------------------------------------------------------------
Returns: 0.856399995066738
Volatility: 0.09493854008923967
Sharpe Ratio: 9.02057261741907
Portfolio Weights: [0.05549935 0.05417608 0.01885594 0.00298535 0.05649435 0.31488822
 0.16272199 0.25708486 0.03096386 0.00626343 0.04006656]
--------------------------------------------------------------------------------

MIN VOLATILITY:
--------------------------------------------------------------------------------
Returns: 0.36320929008347885
Volatility: 0.07225019288204404
Sharpe Ratio: 5.0271047812489
Portfolio Weights: [0.14749584 0.00655644 0.04340685 0.27679508 0.15079983 0.03663565
 0.15501133 0.09160337 0.07666837 0.0144301  0.00059715]
--------------------------------------------------------------------------------


In [15]:
# DICTIONARIES WITH WEIGHTS

maxsharpedict = { "Communication Services": 0.181, "Consumer Discretionary": 0.0458,
                  "Consumer Staples": 0.0105, "Energy": 0.0437, "Financials": 0.0043,
                  "Health Care": 0.0278, "Industrials": 0.0619, "Information Technology": 0.2273,
                  "Materials": 0.1414, "Real Estate": 0.2024,"Utilites": 0.0539 }

minvoldict = { "Communication Services": 0.2683, "Consumer Discretionary": 0.0288,
               "Consumer Staples": 0.235, "Energy": 0.0597, "Financials": 0.007,
               "Health Care": 0.0763, "Industrials": 0.044, "Information Technology": 0.0356,
               "Materials": 0.0141, "Real Estate": 0.034, "Utilites": 0.1973 }